In [ ]:
# Copyright (c) OpenMMLab. All rights reserved.
import random
random.seed(0)
import argparse
import os.path as osp

from mmengine.config import Config, DictAction
from mmengine.utils import ProgressBar

from mmseg.registry import DATASETS, VISUALIZERS
from mmseg.utils import register_all_modules

from mmseg.datasets.transforms import LoadAnnotations_SAL, RandomFlip, RandomCrop, PhotoMetricDistortion, PackSegInputs
from mmcv.transforms import RandomResize,LoadImageFromFile
from mmseg.utils.misc import stack_batch

from mmseg.models.data_preprocessor import SegDataPreProcessor

import numpy as np


In [ ]:
img_path = '/media/ders/mazhiming/mmseg4wsss/mmsegmentation/data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg'
gt_path = '/media/ders/mazhiming/mmseg4wsss/mmsegmentation/data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png'
sal_path = '/media/ders/mazhiming/mmseg4wsss/mmsegmentation/data/VOCdevkit/VOC2012/saliency_map/2007_000032.png'

In [ ]:
class Normalize:
    def __init__(self, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
        self.mean = mean
        self.std = std

    def __call__(self, img):
        img_arr = np.asarray(img)
        normalized_img = np.empty_like(img_arr, np.float32)

        normalized_img[..., 0] = (img_arr[..., 0] / 255. - self.mean[0]) / self.std[0]
        normalized_img[..., 1] = (img_arr[..., 1] / 255. - self.mean[1]) / self.std[1]
        normalized_img[..., 2] = (img_arr[..., 2] / 255. - self.mean[2]) / self.std[2]

        return normalized_img

In [ ]:
randomCrop=RandomCrop(crop_size=(448,448),cat_max_ratio=0.75)
randomFlip=RandomFlip(prob=0.5)
randomResize=RandomResize(ratio_range=(0.5, 2.0),scale=(256,512),keep_ratio=True)
loadImg=LoadImageFromFile()
loadAnn=LoadAnnotations_SAL()
photoMetricDistortion=PhotoMetricDistortion(brightness_delta=77,contrast_range=(0.7,1.3),saturation_range=(0.7,1.3),hue_delta=26)#brightness_delta=77,contrast_range=(0.7,1.3),saturation_range=(0.7,1.3),hue_delta=0.1
packSegInputs=PackSegInputs()
normalize = Normalize()



In [ ]:
results = dict(
    img_path=img_path,
    seg_map_path=gt_path,
    sal_path=sal_path,
    reduce_zero_label=False,
    seg_fields=[])

In [ ]:
def transform_img(results):
    x=loadImg(results)
    x=loadAnn(x)
    x=randomResize(x)
    
    x=randomFlip(x)
    x=photoMetricDistortion(x)#jetter
    x=randomCrop(x)
    x=packSegInputs(x)
    return x

In [ ]:
x=transform_img(results)

In [ ]:
# x['data_samples'].gt_sem_seg.data

In [ ]:
# x['data_samples']

In [ ]:
x['data_samples'].sal_map.data.size()

In [ ]:
x['inputs'].size()

In [ ]:
import matplotlib.pyplot as plt
import torch
img_or=plt.imread(img_path)
plt.imshow(img_or)

In [ ]:
# normalize = Normalize()
# x['inputs'] = normalize(x['inputs'])
plt.imshow(x['inputs'].numpy().transpose(1,2,0)[...,::-1])

In [ ]:
plt.imshow(x['data_samples'].sal_map.data.numpy().transpose(1,2,0)*255)

#stack_batch func

In [ ]:
mean=[123.675, 116.28, 103.53]
std=[58.395, 57.12, 57.375]
pad_val=0
seg_pad_val=255
size=(448,448)
type='SegDataPreProcessor'
dataSamples=x['data_samples']
inputs=x['inputs']

In [ ]:
# outputdata,padded_samples=stack_batch(inputs=[inputs], data_samples=[dataSamples],  size=size)

In [ ]:
# outputdata.max()

In [ ]:
# plt.imshow(outputdata[0].numpy().transpose(1,2,0))
# outputdata.shape

In [ ]:
# padded_samples[0].sal_map.data.size()
# plt.imshow(padded_samples[0].sal_map.data.numpy().transpose(1,2,0))

In [ ]:
# padded_samples[0].sal_map.data.max()
# 

In [ ]:
segDataPpreprocessor=SegDataPreProcessor(bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=0,
    size=(
        448,
        448,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],)

In [ ]:
x['data_samples'].gt_sem_seg.data

In [ ]:
x['inputs']=[x['inputs']]
x['data_samples']=[x['data_samples']]

In [ ]:
input_of_model=segDataPpreprocessor(x,training=True)

In [ ]:
input_of_model['inputs'].shape

In [ ]:
input_of_model['inputs']

In [ ]:
plt.imshow(input_of_model['inputs'][0].numpy().transpose(1,2,0))

In [ ]:
input_of_model['data_samples'].sal_map.data.size()
plt.imshow(input_of_model['data_samples'].sal_map.data.numpy().transpose(1,2,0))